In [1]:
import machine_common_sense as mcs
import pickle
import numpy as np
from tqdm import tqdm
import glob
import time
import jax.numpy as jnp
import PIL
import jax
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
plt.rcParams['animation.ffmpeg_path'] = '/opt/homebrew/bin/ffmpeg'

In [7]:
class MCS_Observation:
    def __init__(self, rgb, depth, intrinsics, segmentation, cam_pose):
        """RGBD Image
        
        Args:
            rgb (np.array): RGB image
            depth (np.array): Depth image
            camera_pose (np.array): Camera pose. 4x4 matrix
            intrinsics (b.camera.Intrinsics): Camera intrinsics
            segmentation (np.array): Segmentation image
        """
        self.rgb = rgb
        self.depth = depth
        self.intrinsics = intrinsics
        self.segmentation  = segmentation
        self.cam_pose = cam_pose

def get_obs_from_step_metadata(step_metadata, intrinsics, cam_pose):
    rgb = np.array(list(step_metadata.image_list)[-1])
    depth = np.array(list(step_metadata.depth_map_list)[-1])
    seg = np.array(list(step_metadata.object_mask_list)[-1])
    colors, seg_final_flat = np.unique(seg.reshape(-1,3), axis=0, return_inverse=True)
    seg_final = seg_final_flat.reshape(seg.shape[:2])
    observation = MCS_Observation(rgb, depth, intrinsics, seg_final, cam_pose)
    return observation

def cam_pose_from_step_metadata(step_metadata):
    cam_pose_diff_orientation = np.array([
        [ 1,0,0,0],
        [0,0,-1,-4.5], # 4.5 is an arbitrary value
        [ 0,1,0,step_metadata.camera_height],
        [ 0,0,0,1]
    ])
    inv_cam_pose = np.linalg.inv(cam_pose_diff_orientation)
    inv_cam_pose[1:3] *= -1
    cam_pose = np.linalg.inv(inv_cam_pose)
    return cam_pose

def intrinsics_from_step_metadata(step_metadata):
    width, height = step_metadata.camera_aspect_ratio
    aspect_ratio = width / height
    cx, cy = width / 2.0, height / 2.0
    fov_y = np.deg2rad(step_metadata.camera_field_of_view)
    fov_x = 2 * np.arctan(aspect_ratio * np.tan(fov_y / 2.0))
    fx = cx / np.tan(fov_x / 2.0)
    fy = cy / np.tan(fov_y / 2.0)
    clipping_near, clipping_far = step_metadata.camera_clipping_planes
    intrinsics = {
        'width' : width,
        'height' : height,
        'cx' : cx,
        'cy' : cy,
        'fx' : fx,
        'fy' : fy,
        'near' : clipping_near,
        'far' : clipping_far
    }
    return intrinsics

scene_folder = 'mcs/val_7/*.json'
files = sorted(glob.glob(scene_folder))
controller = mcs.create_controller("mcs/config_level2.ini")

for i, file in enumerate(files):
    scene_data = mcs.load_scene_json_file(file)
    step_metadata = controller.start_scene(scene_data)
    scene_intrinsics = intrinsics_from_step_metadata(step_metadata)
    scene_cam_pose = cam_pose_from_step_metadata(step_metadata)
    MCS_Observations = [get_obs_from_step_metadata(step_metadata, scene_intrinsics, scene_cam_pose)]

    def MCS_stepper():
        while True:
            yield

    for _ in tqdm(MCS_stepper()):
        step_metadata = controller.step("Pass")
        if len(step_metadata.action_list) == 0:
            break
        MCS_Observations.append(get_obs_from_step_metadata(step_metadata, scene_intrinsics))  # Do stuff here


Found path: /home/ubuntu/.mcs/0.7.3/MCS-AI2-THOR-Unity-App-v0.7.3.x86_64


99it [00:46,  2.11it/s]
199it [01:32,  2.15it/s]


In [3]:
with open(f"final_result_48069476762.pkl", "rb") as file:
    final_result = pickle.load(file)

In [15]:
type(final_result['report']['score'])

KeyError: 'score'

In [20]:
final_result['report'][1]

{'rating': 1, 'score': 1.0}

In [17]:
type(final_result['report'][1]['score'])

float

In [30]:
def haha(x,y,z):
    print(x)
    print()
    print(y)
    print()
    print(z)
haha(*final_result.values())

1

1.0

{1: {'rating': 1, 'score': 1.0}, 2: {'rating': 1, 'score': 1.0}, 3: {'rating': 1, 'score': 1.0}, 4: {'rating': 1, 'score': 1.0}, 5: {'rating': 1, 'score': 1.0}, 6: {'rating': 1, 'score': 1.0}, 7: {'rating': 1, 'score': 1.0}, 8: {'rating': 1, 'score': 1.0}, 9: {'rating': 1, 'score': 1.0}, 10: {'rating': 1, 'score': 1.0}, 11: {'rating': 1, 'score': 1.0}, 12: {'rating': 1, 'score': 1.0}, 13: {'rating': 1, 'score': 1.0}, 14: {'rating': 1, 'score': 1.0}, 15: {'rating': 1, 'score': 1.0}, 16: {'rating': 1, 'score': 1.0}, 17: {'rating': 1, 'score': 1.0}, 18: {'rating': 1, 'score': 1.0}, 19: {'rating': 1, 'score': 1.0}, 20: {'rating': 1, 'score': 1.0}, 21: {'rating': 1, 'score': 1.0}, 22: {'rating': 1, 'score': 1.0}, 23: {'rating': 1, 'score': 1.0}, 24: {'rating': 1, 'score': 1.0}, 25: {'rating': 1, 'score': 1.0}, 26: {'rating': 1, 'score': 1.0}, 27: {'rating': 1, 'score': 1.0}, 28: {'rating': 1, 'score': 1.0}, 29: {'rating': 1, 'score': 1.0}, 30: {'rating': 1, 'score': 1.0}, 31: {'rati

In [18]:
step_metadata.camera_height


'camera_height' in dir(step_metadata)

True

In [17]:
cam_pose_diff_orientation = np.array([
    [ 1,0,0,0],
    [0,0,-1,-4.5], # 4.5 is an arbitrary value
    [ 0,1,0,step_metadata.camera_height],
    [ 0,0,0,1]
])
inv_cam_pose = np.linalg.inv(cam_pose_diff_orientation)
inv_cam_pose[1:3] *= -1
cam_pose = np.linalg.inv(inv_cam_pose)
cam_pose

array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. , -4.5],
       [-0. , -1. , -0. ,  1.5],
       [ 0. ,  0. ,  0. ,  1. ]])

In [26]:
@jax.jit
def tester(x,z):
    y = jax.lax.cond(jnp.greater(x, 5),lambda:1, lambda:5)
    if z.shape[0] == 3:
        w = 3
    else:
        w = 9
    return y,w



In [27]:
tester(3,jnp.ones(43))

(Array(5, dtype=int32, weak_type=True), Array(9, dtype=int32, weak_type=True))

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/physics_conda/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/miniconda3/envs/physics_conda/lib/python3.9/threading.py", line 1306, in run
    self.function(*self.args, **self.kwargs)
  File "/home/ubuntu/miniconda3/envs/physics_conda/lib/python3.9/site-packages/machine_common_sense/controller.py", line 190, in _check_step_for_timeout
    self.end_scene(rating=None, score=-1)
  File "/home/ubuntu/miniconda3/envs/physics_conda/lib/python3.9/site-packages/machine_common_sense/controller.py", line 536, in end_scene
    raise RuntimeError("end_scene called twice with the same scene")
RuntimeError: end_scene called twice with the same scene


: 